# Transfer Learning

Leverage proven working existing Neural Network Architecture to solve problems

Tune architectures to our specific needs

Feature Extraction and Fine Tuning

### Feature Extraction

Re-use a pre-trained model as is and append layers if necessary

### Fine-tuning

Re-use a pre-trained model as is and train it against our data changing desired parameters


### Use Tensorboard to track iterations


###### Setup Tensorboard

```
def tensorboard_callback(dir_name, exp_name):
  log_dir = dir_name + '/' + exp_name + '/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
  tensorboard_callback = TensorBoard(log_dir=log_dir)
  print(f'saving tensorboard log files to log dir: {log_dir}')
  return tensorboard_callback
```

Google Colab

`
!tensorboard dev upload --logdir ./tensorflow_hub/ \
--name 'Name of Tensorboard' \
--description 'Description of the Tensorboard' \
--one_shot
`

Show the TensorBoards that are active

`!tensorboard dev list`

Delete the TensorBoard Instances

`!tensorboard dev delete --experiment EXP_ID`
  
### TF Hub

https://tfhub.dev/
  
For image classification there are 2 main categories, Feature Vector and Classification

###### A few of the top Pre-Trained models for Image Classification

- VGG-16
 - comes with TensorFlow
 - `from tensorflow.keras.applications.vgg16 import VGG16`
- ResNet50
- Inceptionv3
- EfficientNet

###### Classification

Binary Classification

##### Feature Vector

Multi-Category Classification


###### Resources to research best Transfer learning models

https://paperswithcode.com/sota/image-classification-on-imagenet

###### Sites to get data images and samples

https://www.cs.toronto.edu/~kriz/cifar.html

http://yann.lecun.com/exdb/mnist/

http://image-net.org/download

https://data.vision.ee.ethz.ch/cvl/datasets_extra/food-101/




### Build the Model

###### Trainable vs Non-Trainable
- Trainable means that the weights of the neurons within the hidden layers will change
- Non-Trainable means that the weights of the neurons within the hidden layers will not change and remain static as they were when they were finally trained on

The default of any transfer learning model is to allow the weights to change BUT we don't want that

Initially, when first using a Transfer Learning Model we want to get a baseline of the model first, therefore we need to set the trainable parameter to false...this process isn't so easy and rather annoying but...

`
for layer in base_model.layers:
    layer.trainable = False
`
Generally we want to keep trainable as false initially to test against the main and then set trainable to true on select layers


###### VGG16

```
base_model = VGG16(input_shape = (256, 256, 3), include_top = False, weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False

#Add Flatten the output layer to convert to a 1 dimensional array
x = layers.Flatten()(base_model.output)

#Optional Layer but you will see this in most other CNN's
x = layers.Dense(512, activation='relu')(x)

#Add a final sigmoid/softmax layer with X node for classification output
x = layers.Dense(1, activation='sigmoid')(x)

#Put it all together in a model combining the base_model and x
model = tf.keras.models.Model(base_model.input, x)

#Compile using the correct loss function
model.compile(optimizer='adam', loss='binary_crossentropy',metrics = ['accuracy'])
```

###### Model from TF Hub Directly

Binary Classification

`
model = Sequential([
    hub.KerasLayer(
      model_url,
      trainable=False,
      input_shape=(256,256,3))
])
model.compile(optimizer='adam', loss='binary_crossentropy',metrics = ['accuracy'])
`

Categorical Classification

`
model = Sequential([
    hub.KerasLayer(
      model_url,
      trainable=False,
      input_shape=(256,256,3)),
    Dense(num_categories, activation='softmax', name='output_layer') 
])
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics = ['accuracy'])
`


In [10]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

from datetime import datetime

In [2]:
df = pd.read_csv('/Users/roadblock/Downloads/TensorFlow_FILES/DATA/cancer_classification.csv')

In [3]:
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,benign_0__mal_1
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [5]:
pwd

'/Users/roadblock/coding/coding101/data_sciences'

In [8]:
board = TensorBoard(
    log_dir='logs/fit',
    histogram_freq=1,
    write_graph=True,
    write_images=True,
    update_freq='epoch',
    profile_batch=2,
    embeddings_freq=1
)

2022-06-11 17:02:39.340581: I tensorflow/core/profiler/lib/profiler_session.cc:99] Profiler session initializing.
2022-06-11 17:02:39.340598: I tensorflow/core/profiler/lib/profiler_session.cc:114] Profiler session started.
2022-06-11 17:02:39.340952: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session tear down.


In [9]:
model = Sequential()
model.add(Dense(30, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(15, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy')

2022-06-11 17:04:19.045300: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
X = df[df.columns[:-1]]
y = df['benign_0__mal_1']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [12]:
scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [14]:
early_stopping = EarlyStopping(monitor='val_loss', mode='min', patience=25)

model.fit(X_train, y_train, epochs=600, callbacks=[early_stopping, board], validation_data=(X_test, y_test), verbose=1)



Epoch 1/600
 1/13 [=>............................] - ETA: 6s - loss: 0.6850

2022-06-11 17:07:41.219697: I tensorflow/core/profiler/lib/profiler_session.cc:99] Profiler session initializing.
2022-06-11 17:07:41.219711: I tensorflow/core/profiler/lib/profiler_session.cc:114] Profiler session started.
2022-06-11 17:07:41.222222: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-06-11 17:07:41.223443: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session tear down.
2022-06-11 17:07:41.225400: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/fit/plugins/profile/2022_06_11_17_07_41

2022-06-11 17:07:41.226158: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/fit/plugins/profile/2022_06_11_17_07_41/roadblocks-MacBook-Pro.local.trace.json.gz
2022-06-11 17:07:41.229819: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/fit/plugins/profile/2022_06_11_17_07_41

2022-06-11 17:07:41.

13/13 [==============================] - 1s 20ms/step - loss: 0.6774 - val_loss: 0.6509
Epoch 2/600
13/13 [==============================] - 0s 8ms/step - loss: 0.6681 - val_loss: 0.6195
Epoch 3/600
13/13 [==============================] - 0s 7ms/step - loss: 0.6504 - val_loss: 0.5945
Epoch 4/600
13/13 [==============================] - 0s 9ms/step - loss: 0.6226 - val_loss: 0.5695
Epoch 5/600
13/13 [==============================] - 0s 8ms/step - loss: 0.6039 - val_loss: 0.5404
Epoch 6/600
13/13 [==============================] - 0s 8ms/step - loss: 0.6041 - val_loss: 0.5117
Epoch 7/600
13/13 [==============================] - 0s 8ms/step - loss: 0.5465 - val_loss: 0.4796
Epoch 8/600
13/13 [==============================] - 0s 8ms/step - loss: 0.5506 - val_loss: 0.4462
Epoch 9/600
13/13 [==============================] - 0s 8ms/step - loss: 0.5185 - val_loss: 0.4137
Epoch 10/600
13/13 [==============================] - 0s 7ms/step - loss: 0.5180 - val_loss: 0.3861
Epoch 11/600
13/13 [